# Wellbeing Police

From various subreddit, we have scraped close to 4000 posts. The subreddits include "r/SuicideWatch", "r/BipolarReddit", "r/Anxiety", "r/AnxietyDepression", "r/Depression", and "r/Happy". Each of the subreddit posts currently sitting within its own csv file. We will see what are the fields that are consistent across the CSV files and choose those applicable. Clean up the data, removed the columns not needed. 

We will proceed to label them according to the subreddit they came from and combine them into a single dataframe.

In [7]:
import pandas as pd
import glob
import os

In [14]:
path = "./reddit/csvs/"
all_csvs = glob.glob(os.path.join(path, "*.csv"))

for filename in all_csvs:
    print(filename)

./reddit/csvs/230305_BipolarReddit_1000_body_text.csv
./reddit/csvs/230304_Depression_new_text_only.csv
./reddit/csvs/230305_anxiety.csv
./reddit/csvs/230305_Happy_1000.csv
./reddit/csvs/230301_SuicideWatch_1000.csv
./reddit/csvs/230304_Depression_new.csv
./reddit/csvs/230305_anxiety_depression.csv


As all the scrapes was created from the same script, the columns of the resulting csv are same across the files. We can directly combine them.

In [25]:
anxiety_df = pd.read_csv("./reddit/csvs/230305_anxiety.csv", index_col = None, header=0)

In [26]:
anxiety_df.head()

,Unnamed: 0,title,score,id,subreddit,url,num_comments,body,created
0,0,Set your intention,14,11cfc89,Anxiety,https://www.reddit.com/r/Anxiety/comments/11cf...,11,Happy Sunday /r/Anxiety!\n\nIt's everyone's fa...,1.677413e+09
1,1,Monthly Check-In Thread,19,118vbge,Anxiety,https://www.reddit.com/r/Anxiety/comments/118v...,44,Hello everyone! Welcome to the r/Anxiety month...,1.677064e+09
2,2,Am I being over dramatic?,41,11im5uf,Anxiety,https://www.reddit.com/r/Anxiety/comments/11im...,35,I haven’t talked to my boyfriend much after we...,1.677987e+09
3,3,I desperately need a break from my own mental ...,370,11i24md,Anxiety,https://www.reddit.com/r/Anxiety/comments/11i2...,81,I am so sick and tired of constantly ruminatin...,1.677944e+09
4,4,Internal Tremors?,58,11igyst,Anxiety,https://www.reddit.com/r/Anxiety/comments/11ig...,33,Does anybody else have severe anxiety and inte...,1.677973e+09


In [24]:
anxiety_depression_df = pd.read_csv("./reddit/csvs/230305_anxiety_depression.csv", index_col = None, header=0)

In [27]:
anxiety_depression_df.head()

,Unnamed: 0,title,score,id,subreddit,url,num_comments,body,created
0,0,this is the end for me,11,11iaboj,AnxietyDepression,https://www.reddit.com/r/AnxietyDepression/com...,7,I might actually get pushed back a year. My re...,1.677957e+09
1,1,Constantly exhausted.,1,11in0xu,AnxietyDepression,https://www.reddit.com/r/AnxietyDepression/com...,0,I started Ambilify for my depression 2 weeks a...,1.677989e+09
2,2,"Medication tolerance in long time users, pleas...",3,11id0m0,AnxietyDepression,https://www.reddit.com/r/AnxietyDepression/com...,18,TLDR: I think my body formed a tolerance to We...,1.677963e+09
3,3,How do I figure out what my redeemable qualiti...,3,11icbsb,AnxietyDepression,https://www.reddit.com/r/AnxietyDepression/com...,1,I don’t know how to convince myself that even...,1.677962e+09
4,4,Feeling unsafe in the world,4,11i9v7f,AnxietyDepression,https://www.reddit.com/r/AnxietyDepression/com...,1,"\n\nHellluuu, \nmy name is Maggie and I'm 20 y...",1.677956e+09


In [29]:
df = pd.concat([anxiety_depression_df, anxiety_df])
df[987:1100]

,Unnamed: 0,title,score,id,subreddit,url,num_comments,body,created
14,14,Why Do I Need Pressure to Calm Me Down?,3,11ioiep,Anxiety,https://www.reddit.com/r/Anxiety/comments/11io...,2,"I have really bad anxiety, but I find pushing ...",1.677994e+09
15,15,Can iPhone be tracked after reset,3,11iq8g5,Anxiety,https://www.reddit.com/r/Anxiety/comments/11iq...,0,So I went to go buy an iPhone off Facebook mar...,1.677999e+09
16,16,"I cannot find peace. I am so scared now, all t...",6,11ik8z7,Anxiety,https://www.reddit.com/r/Anxiety/comments/11ik...,4,The last year for me has been difficult and I'...,1.677981e+09
17,17,Lifelong Condition,5,11inlo2,Anxiety,https://www.reddit.com/r/Anxiety/comments/11in...,5,My anxiety has been with me since I was a youn...,1.677991e+09
18,18,How do I get a life,8,11iihyd,Anxiety,https://www.reddit.com/r/Anxiety/comments/11ii...,4,"[30] I’ve always dealt with a lot of anxiety, ...",1.677977e+09
...,...,...,...,...,...,...,...,...,...
122,122,Those Who Came Off Benzos,3,11i071p,Anxiety,https://www.reddit.com/r/Anxiety/comments/11i0...,7,How long until you felt recovered? Almost 3 m...,1.677939e+09
123,123,I have to give a presentation in class and my ...,1,11ibu7r,Anxiety,https://www.reddit.com/r/Anxiety/comments/11ib...,0,I'm in grad school and I have really bad anxie...,1.677961e+09
124,124,Citalopram,1,11ibsnk,Anxiety,https://www.reddit.com/r/Anxiety/comments/11ib...,1,I’ve been taking citalopram since I was a teen...,1.677961e+09
125,125,question,2,11i3ukg,Anxiety,https://www.reddit.com/r/Anxiety/comments/11i3...,1,I've had pretty crippling social and regular a...,1.677947e+09
